# Matrix Factorization using Gradient Descent

In this excercise, you are required to implement matrix factorization method, specifically [Non-Negative Matrix Factorization (NMF)](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization), using gradient descent. You have to apply the matrix factorization to solve topic modeling. 

(Please refer to the tutorial on basics of topic modeling, LSI with SVD (Tutorial Set 4), for details on LSI)

## Applying NMF to solve Topic Modeling
Given a term document matrix $V$, NMF factorizes it into two matrix $W$ and $H$ with the property that all three documents have no negative elements.
<img src="content/NMF.png" alt="Non-negative matrix factorization" style="width: 80%">

In Non-negative Matrix Factorization, a document-term matrix is approximately factorized into term-feature and feature-document matrices.

$V = WH$ Matrix multiplication can be implemented as computing the column vectors of $V$ as linear combinations of the basis vectors (column vectors) in $W$ (or the topics discovered from the documents) using coefficients supplied by columns of $H$ (or the membership weights for the topics in each document). That is, each column of V can be computed as follows:
$$ v_i = W h_i$$

In what follows, we will first see an example of applying NMF by using [SKLearn NMF API](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) for the task of topic modeling. Later you will be required to implement NMF using gradient descent.

### Scikit-Learn implementation of NMF for topic modeling
Given a set of multivariate  $n$-dimensional data vectors, they are put into an  $n\times m$  matrix  $V$  as its columns, where  $m$  is the number of examples in the data set. This matrix  $V$  is approximately factorized into an  $n \times t$  matrix  $W$  and an  $t \times m$  matrix  $H$ , where  $t$  is generally less than  $n$  or  $m$ . Hence, this results in a compression of the original data matrix.

In terms of topic modeling, the input document-term matrix  $V$  is factorized into a  $n \times t$  document-topic matrix and a  $t \times m$  topic-term matrix, where  $t$  is the number of topics produced. Similar to tutorial 4, we will be using 20 NewsFetch dataset for the task.

#### Imports

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

%matplotlib inline

#### Setup data

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)


#### Compute document features

In [3]:
vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectors_tfidf = vectorizer_tfidf.fit_transform(newsgroups_train.data).todense() # (documents, vocab)
vectors_tfidf.shape

(2034, 26576)

#### Compute NMF using Scikit Learn library

We will also write a function to display top 8 words for each topic.

In [4]:
num_top_words=8
vocab = np.array(vectorizer_tfidf.get_feature_names())

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [5]:
from sklearn import decomposition

d = 5 # num topics
clf = decomposition.NMF(n_components=d, random_state=1)

In [6]:
W1 = clf.fit_transform(vectors_tfidf)
H1 = clf.components_

In [7]:
show_topics(H1)

['people don think just like objective say morality',
 'graphics thanks files image file program windows know',
 'space nasa launch shuttle orbit moon lunar earth',
 'ico bobbe tek beauchaine bronx manhattan sank queens',
 'god jesus bible believe christian atheism does belief']

# Excercise

## NMF using SGD

In stochastic gradient descent (SGD), we evaluate our loss function on just a sample of our data (sometimes called a mini-batch). We would get different loss values on different samples of the data, so this is why it is stochastic. It turns out that this is still an effective way to optimize, and it's much more efficient!

### Applying SGD to NMF

Goal: Decompose $V\;(m \times n)$ into
$$ V \approx HW$$
where $W\;(m \times d)$ and $H\;(d \times n)$, $W,\;H\; \geq \;0$, and we've minimized the Frobenius norm of $V-WH$. The objective function can therefore be written as the following:
$$
\min_{H \geq 0, W \geq 0} F(H,W) = \frac{1}{2} ||V - HW||^{2} + \frac{\lambda}{2} \left( ||H||^2 + ||W||^2 \right)
$$

### Implementation of NMF using SGD (Excercise)
__Approach:__ Given the objective function above, pick random positive $W$ & $H$, and then use SGD to optimize. 

(Note that the objective function is non-convex in nature, and is convex only if we consider $H$ and $W$ separately. You can directly write the gradient descent rule for the objective function presented above)



In [8]:
#Function for calculating the value of forbenius norm objective function 
def lossfunc(Mat,W,H,lamda):
    Mat_pred=np.matmul(W,H)
    loss = 0.5 * np.square(np.linalg.norm(Mat-Mat_pred)) + 0.5 * lamda * (np.square(np.linalg.norm(W)) + np.square(np.linalg.norm(Mat-Mat_pred)))
    return(loss)

In [9]:
def grads(Mat,W,H,lamda):
    grad_w= np.matmul((np.matmul(W,H) - Mat), H.T) + lamda*W
    grad_h= np.matmul(W.T, (np.matmul(W,H) - Mat)) + lamda*H
    return(grad_w,grad_h)

In [12]:
''' WRITE YOUR CODE BELOW '''
def sgd(Mat,num_topics,eta=0.1,lamda=0.5,epsilon=1e-05):
    #The input document-term matrix Mat mxn is factorized into a  m×𝑡  document-topic matrix(W)
    #and a  𝑡×n  topic-term matrix(H), where t is the number of topics produced using SGD
    i=0
    Error2=0
    w_history = []
    h_history = []
    loss_history = []
    m,n =Mat.shape
    # Initializing weights
    W = np.abs(np.random.normal(scale=0.01, size=(m,num_topics)))
    H = np.abs(np.random.normal(scale=0.01, size=(num_topics,n)))
    # Applying SGD
    while(1):
        w_history.append(W)
        h_history.append(H)
        w_k=W
        h_k=H
        # Loss and Gradient calculaation
        Loss= lossfunc(Mat,W,H,lamda)
        loss_history.append(Loss)
        grad_w,grad_h= grads(Mat,W,H,lamda) 
        #update
        W = W - eta*grad_w
        H = H - eta*grad_h
        i+=1
        Error1=Error2
        Error2 = np.abs(np.mean(Mat -W@H))
        print('Error in iteration',i,'is',Error2)
        if  (np.abs(Error1-Error2)) < epsilon: #If error is not decreasing more than epsilon in 2 consecutive iterations
            break
    return(W,H,loss_history,h_history,w_history,i,Error2) 



In [13]:
Mat=vectors_tfidf
num_topics = 5
W_mat,H_mat,loss_history,h_history,w_history,num_iterations,Error = sgd(Mat,num_topics,eta=0.01,lamda=0.5,epsilon=5e-06)

print ('Final Error',Error,'\n','Loss:',loss_history,'\n','Total number of iterations before convergence:',num_iterations)

Error in iteration 1 is 8.562116888746226e-05
Error in iteration 2 is 7.26244502316781e-05
Error in iteration 3 is 6.178388829927012e-05
Error in iteration 4 is 5.277025363245141e-05
Error in iteration 5 is 4.5315801024460875e-05
Error in iteration 6 is 3.92002458590754e-05
Error in iteration 7 is 3.424053011063114e-05
Final Error 3.424053011063114e-05 
 Loss: [1480.4515720763334, 1479.882280976941, 1479.4174072549167, 1479.0301763231987, 1478.7010158296341, 1478.4154197870255, 1478.1625082362243] 
 Total number of iterations before convergence: 7


In [14]:
print (Mat.shape,W_mat.shape,H_mat.shape,'Mat-H_mat*W_mat=',Error)

(2034, 26576) (2034, 5) (5, 26576) Mat-H_mat*W_mat= 3.424053011063114e-05
